In [ ]:
#Deep-Genomics

In [3]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
from astropy.io import fits

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

%matplotlib inline

In [4]:
path = r'/home/rbhat/genome/final_output.txt' #genome
#path = r'/home/rbhat/genome/newfileoutput_small.txt' #genome 

In [5]:
genomeData = pd.read_csv(path,sep='~~')

In [6]:
genomeData.shape

(1008, 496757)

In [7]:
#genomeData.iloc[8,:].shape
genomeData.iloc[8:10,:]
#genomeData.ix[8:10,:].head()

,#CHROM,chrM,chrM.1,chrM.2,chrM.3,chrM.4,chrM.5,chr1,chr1.1,chr1.2,chr1.3,chr1.4,chr1.5,chr1.6,chr1.7,chr1.8,chr1.9,chr1.10,chr1.11,chr1.12,
8,S1,"0/0:137,0:137:99:0,412,5286","0/0:135,0:135:99:0,406,5208","0/0:128,0:128:99:0,385,4938","0/0:125,0:125:99:0,376,4925","0/0:124,0:124:99:0,373,4239","0/0:208,0:208:99:0,626,7956","0/0:252,0:252:99:0,758,9047","1/1:0,294:294:99:10449,885,0","0/0:294,0:294:99:0,885,10952","0/0:254,0:254:99:0,761,9604","0/0:215,0:215:99:0,647,7911","0/0:14,0:14:42:0,42,516","0/0:27,0:27:81:0,81,1000","0/0:46,0:46:99:0,138,1791","0/0:55,0:55:99:0,166,2122","0/0:45,0:45:99:0,135,1684","0/0:35,0:35:99:0,105,1356","0/0:30,0:30:90:0,90,1084","0/0:28,0:28:84:0,84,1006",...
9,S2,"0/0:151,0:151:99:0,454,5807","0/0:149,0:149:99:0,448,5748","0/0:147,0:147:99:0,442,5652","0/0:149,0:149:99:0,448,5794","0/0:127,0:127:99:0,382,4309","0/0:201,0:201:99:0,605,7649","0/0:223,0:223:99:0,671,8172","1/1:4,237:237:99:8278,701,0","0/0:170,0:170:99:0,502,6128","0/0:201,6:201:99:0,602,7416","0/0:181,0:181:99:0,530,6494","0/0:28,0:28:84:0,84,1034","0/0:42,0:42:99:0,126,1587","0/0:41,0:41:99:0,123,1533","0/0:44,0:44:99:0,132,1708","0/0:49,0:49:99:0,147,1839","0/0:36,0:36:99:0,108,1440","0/0:34,0:34:99:0,102,1199","0/0:21,0:21:63:0,63,756",...


In [8]:
label_path = r'/home/rbhat/genome/4-Bipolar_exomes_training.txt' #genome label
genomeDataLabel = pd.read_csv(label_path,sep='\t')

In [9]:
print(genomeDataLabel.shape)
#print(genomeDataLabel.iloc[0,:])
print(genomeDataLabel.head())

(500, 2)
  individual  disease_status
0         S1               0
1         S2               0
2         S3               0
3         S4               0
4         S5               0

[5 rows x 2 columns]


In [10]:
X_train = genomeData[8:408]
y_train = genomeDataLabel[0:400]
print(X_train.shape, y_train.shape)

X_test = genomeData[409:508]
y_test = genomeDataLabel[401:500]
#print(X_train.tail())
#print(Y_train.tail())
print(X_test.shape, y_test.shape)
#print(X_test.tail())
#print(Y_test.tail())

# the data, shuffled and split between tran and test sets
#(X_train, y_train), (X_test, y_test) = mnist.load_data()

((400, 496757), (400, 2))
((99, 496757), (99, 2))


In [11]:
X_train1 = X_train
X_test1 = X_test

In [12]:
def clean(x):
    x=str(x)
    if ':' in x:
        x=x.split(':')[0]
        if x=='0/0':
            x = 1
            x = float(x)
        elif x == '0/1':
            x = 2
            x = float(x)
        elif x == '1/1':
            x = 3
            x = float(x)
        else:
            x = float(0)                  
    return x


print(clean('1/1:137,0:137:99:0,412,5286'))
#X_train1 = X_train1.apply(lambda row:)

3.0


In [13]:
#print(X_train1.shape, X_test1.shape)
X_train2 = X_train1.applymap(clean)
X_test2 = X_test1.applymap(clean)

In [15]:
#print(X_train2.head())
#print(X_test2.head())
X_train3 = X_train2.iloc[:,1:]
X_test3 = X_test2.iloc[:,1:]
#print(X_train3.shape)
print(X_test3.head())

     chrM  chrM.1  chrM.2  chrM.3  chrM.4  chrM.5  chr1  chr1.1  chr1.2  \
409     1       1       1       1       1       1     1       3       1   
410     1       1       1       1       1       1     1       3       1   
411     1       1       1       1       1       1     1       3       1   
412     1       1       1       1       1       1     1       3       1   
413     1       1       1       1       1       1     1       3       1   

     chr1.3  chr1.4  chr1.5  chr1.6  chr1.7  chr1.8  chr1.9  chr1.10  chr1.11  \
409       1       1       1       1       1       1       1        1        1   
410       1       1       1       1       1       1       1        1        1   
411       1       1       1       1       1       1       1        1        1   
412       1       1       1       1       1       1       1        1        1   
413       1       1       1       1       1       1       1        1        1   

     chr1.12  chr1.13      
409        1        1 ...  
410   

In [18]:
#X_train3.shape
#X_test3

In [20]:
#X_train3.to_csv('/home/rbhat/genome/pca_data_train.csv')
#X_test3.to_csv('/home/rbhat/genome/pca_data_test.csv')

#print(X_train.shape, y_train.shape)
#type(X_train)
#print(X_train.iloc[0,:])
print(X_train3.iloc[0,:])
#print(X_test2.iloc[0,:])

chrM      1
chrM.1    1
chrM.2    1
chrM.3    1
chrM.4    1
chrM.5    1
chr1      1
chr1.1    3
chr1.2    1
chr1.3    1
chr1.4    1
chr1.5    1
chr1.6    1
chr1.7    1
chr1.8    1
...
chr10.16612    1
chr10.16613    1
chr10.16614    1
chr10.16615    1
chr10.16616    1
chr10.16617    1
chr10.16618    1
chr10.16619    1
chr10.16620    1
chr10.16621    1
chr10.16622    1
chr10.16623    1
chr10.16624    1
chr10.16625    1
chr10.16626    1
Name: 8, Length: 496756, dtype: float64


In [103]:
#X_train = X_train.reshape(60000, 784)
#X_test = X_test.reshape(10000, 784)
#X_train = X_train.astype("float32")
#X_test = X_test.astype("float32")
#X_train /= 255
#X_test /= 255
#print(X_train.shape[0], 'train samples')
#print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
#print(nb_classes)
#print(y_train.iloc[:,1])

batch_size = 30
nb_classes = 2
nb_epoch = 2
#nb_epoch = 10

#np_utils.to_categorical(y_train.iloc[:,1])

Y_train = np_utils.to_categorical(y_train.iloc[:,1], nb_classes)
Y_test = np_utils.to_categorical(y_test.iloc[:,1], nb_classes)

#Y_train = np_utils.to_categorical(y_train.iloc[:,1])
#Y_test = np_utils.to_categorical(y_test.iloc[:,1])

In [88]:
#print(y_test.head())
#print(Y_test)
print(type(X_train3.as_matrix))
print(type(Y_train))

print(X_train3.shape)
print(Y_train.shape)

print(X_test3.shape)
print(Y_test.shape)

<type 'instancemethod'>
<type 'numpy.ndarray'>
(400, 496756)
(400, 2)
(99, 496756)
(99, 2)


In [89]:
X_train4 = X_train3
X_test4  = X_test3

#X_train4 = X_train3.iloc[:,0:124189]
#X_test4  = X_test3.iloc[:,0:124189]

print(X_train4.shape,X_test4.shape)
496756/4

((400, 496756), (99, 496756))


124189

In [90]:
print(type(X_train4.as_matrix()))
print(X_train4.as_matrix())

<type 'numpy.ndarray'>
[[ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 ..., 
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]
 [ 1.  1.  1. ...,  1.  1.  1.]]


In [91]:
print(X_train4.shape)
print(Y_train.shape)

(400, 496756)
(400, 2)


In [138]:
X_train_log = genomeData[8:508]
y_train_log = genomeDataLabel[0:500]
print(X_train_log.shape, y_train_log.shape)

X_test_log = genomeData[508:]
#y_test_log = genomeDataLabel[400:500]
print(X_test_log.shape)

((500, 496757), (500, 2))
(500, 496757)


In [139]:
#print(X_train1.shape, X_test1.shape)
X_train2_log = X_train_log.applymap(clean)
X_test2_log = X_test_log.applymap(clean)

In [141]:
X_train3_log = X_train2_log.iloc[:,1:]
X_test3_log = X_test2_log.iloc[:,1:]


In [143]:
X_test3_log.head()

,chrM,chrM.1,chrM.2,chrM.3,chrM.4,chrM.5,chr1,chr1.1,chr1.2,chr1.3,chr1.4,chr1.5,chr1.6,chr1.7,chr1.8,chr1.9,chr1.10,chr1.11,chr1.12,chr1.13,
508,1,1,1,1,1,1,1,2,1,2,1,1,1,1,1,1,1,1,1,1,...
509,1,1,1,1,1,1,1,2,1,2,1,1,1,1,1,1,1,1,1,1,...
510,1,1,1,1,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,...
511,1,1,1,1,1,1,1,2,1,2,1,1,1,1,1,1,1,1,1,1,...
512,1,1,1,1,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,...


In [144]:
#Logistic Regression
#Logistic Regression-sklearn
logReg = LogisticRegression()
logReg.fit(X_train3_log.as_matrix(),y_train_log.iloc[:,1])
Y_logPred      = logReg.predict(X_test3_log.as_matrix())
Y_logPred_prob = logReg.predict_proba(X_test3_log.as_matrix())
#print metrics.accuracy_score(y_test_log.iloc[:,1],Y_logPred)

In [161]:
#(1-np.std(Y_logPred_prob,axis=1))

In [157]:
Y_logPred.tofile('/home/rbhat/genome/Y_logPred.txt',sep=',')

In [158]:
#Writing file
(1-np.std(Y_logPred_prob,axis=1)).tofile('/home/rbhat/genome/Y_logPred_prob.txt',sep=',')

In [111]:
#Confusion matrix of logistic regression
conf = metrics.confusion_matrix(y_test.iloc[:,1],Y_logPred)
#plt.imshow(conf)
print conf

[[32 20]
 [13 34]]


In [112]:
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1']

print(classification_report(y_test.iloc[:,1], Y_logPred, target_names=target_names))

             precision    recall  f1-score   support

    class 0       0.71      0.62      0.66        52
    class 1       0.63      0.72      0.67        47

avg / total       0.67      0.67      0.67        99



In [116]:
Y_logPred_prob = logReg.predict_proba(X_test4.as_matrix())

In [125]:
print(Y_logPred_prob[0:5])
print(Y_logPred[0:5])
print(1-np.std(Y_logPred_prob[2]))

[[ 0.59889115  0.40110885]
 [ 0.80884916  0.19115084]
 [ 0.75639366  0.24360634]
 [ 0.32090952  0.67909048]
 [ 0.87521774  0.12478226]]
[0 0 0 1 0]
0.743606342082


In [113]:
#np.array(y_train.iloc[:,1])
#Y_train
#Y_test

In [108]:

model = Sequential()
model.add(Dense(1000, input_dim=496756))
#model.add(Dense(500, input_dim=124189))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('sigmoid'))
#model.add(Activation('softmax'))

#model.compile(loss='categorical_crossentropy', optimizer='sgd', class_mode='binary')
model.compile(loss='categorical_crossentropy', optimizer='adam', class_mode='categorical')
#model.compile(loss='categorical_crossentropy', optimizer='adam', class_mode='categorical')

#model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=True, verbose=2, validation_data=(X_test, Y_test))
#score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)

#model.fit(X_train4.as_matrix(), np.array(y_train.iloc[:,1]), batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=True, verbose=2, validation_data=(X_test4.as_matrix(), np.array(y_test.iloc[:,1])) )  
#score = model.evaluate(X_test4.as_matrix(), np.array(y_test.iloc[:,1]), show_accuracy=True, verbose=0)


model.fit(X_train4.as_matrix(), Y_train, batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=True, verbose=2, validation_data=(X_test4.as_matrix(), Y_test))
score = model.evaluate(X_test4.as_matrix(), Y_test, show_accuracy=True, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 400 samples, validate on 99 samples
Epoch 1/2
133s - loss: 4.4828 - acc: 0.4925 - val_loss: 7.6520 - val_acc: 0.5253
Epoch 2/2
125s - loss: 8.0993 - acc: 0.4975 - val_loss: 7.6520 - val_acc: 0.5253
('Test score:', 7.6520252227783203)
('Test accuracy:', 0.5252525252525253)


In [109]:
#score
preds = model.predict_classes(X_test4.as_matrix(), verbose=0)
print(preds.shape)
print(preds)

(99,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [110]:
#print(X_test4.shape)
valid_preds = model.predict_proba(X_test4.as_matrix(), verbose=0)
print(valid_preds)

[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]

In [ ]:
#Calculating ROC
'''
valid_preds = model.predict_proba(X_valid, verbose=0)
valid_preds = valid_preds[:, 1]	128	    valid_preds = valid_preds[:, 1]
roc = metrics.roc_auc_score(y_valid, valid_preds)	129	    roc = metrics.roc_auc_score(y_valid, valid_preds)
print("ROC:", roc)	130	    print("ROC:", roc)
av_roc += roc	131	    av_roc += roc
print('Average ROC:', av_roc/nb_folds)	133	print('Average ROC:', av_roc/nb_folds)
'''

In [ ]:
'''
batch_size = 10
nb_classes = 2
nb_epoch = 10

X_train = genomeData[8:408]
y_train = genomeDataLabel[0:400]
print(X_train.shape, y_train.shape)

X_test = genomeData[409:508]
y_test = genomeDataLabel[401:500]
#print(X_train.tail())
#print(Y_train.tail())
print(X_test.shape, y_test.shape)
#print(X_test.tail())
#print(Y_test.tail())

# the data, shuffled and split between tran and test sets
#(X_train, y_train), (X_test, y_test) = mnist.load_data()

batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

# the data, shuffled and split between tran and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta')

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          show_accuracy=True, verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])
'''